In [1]:
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.interpolate import interp1d
from scipy.signal import welch
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import datetime as dt
import threading
import time

import BrainBitPython.BrainBitDemo.neuro_impl.emotions_bipolar_controller
import BrainBitPython.BrainBitDemo.neuro_impl.emotions_monopolar_controller
import BrainBitPython.BrainBitDemo.neuro_impl.spectrum_controller

from BrainBitPython.BrainBitDemo.neuro_impl.emotions_bipolar_controller import EmotionBipolar
from BrainBitPython.BrainBitDemo.neuro_impl.emotions_monopolar_controller import EmotionMonopolar
from BrainBitPython.BrainBitDemo.neuro_impl.spectrum_controller import SpectrumController

importlib.reload(BrainBitPython.BrainBitDemo.neuro_impl.emotions_bipolar_controller)
importlib.reload(BrainBitPython.BrainBitDemo.neuro_impl.emotions_monopolar_controller)
importlib.reload(BrainBitPython.BrainBitDemo.neuro_impl.spectrum_controller)

import hrvanalysis

mpl.rcParams['animation.ffmpeg_path'] = r'C:\Program Files\FFmpeg\bin\ffmpeg.exe'

%matplotlib widget

In [2]:
print(animation.writers.list())

['pillow', 'ffmpeg', 'ffmpeg_file', 'html']


In [3]:
LiveChartWindowSizeInSeconds = 60

HeartRate_CSVFileName = 'Checkme O2 Max _20250716192137'
HeartRateMeasuringIntervalInSeconds = 2

EEG_CSVFileName = '15.07.2025_17-42 - 15.07.2025_17-42'

ChartExportFileName = "Brett Chart"

In [4]:
df = pd.read_csv(f'Data/Watch/{HeartRate_CSVFileName}.csv')

# Remove non-numeric pulse entries if needed
df = df[df['Pulse Rate'] != '--']
df['Pulse Rate'] = pd.to_numeric(df['Pulse Rate'], errors='coerce')
df.dropna(subset=['Pulse Rate'], inplace=True)

# Convert Time column to datetime
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S %b %d %Y')

# Convert BPM to RR intervals in milliseconds
# RR(ms) = (60 / HR)
df['RR'] = 60 / df['Pulse Rate']

def compute_rmssd(rr_intervals):
    diffs = np.diff(rr_intervals)
    squared_diffs = diffs ** 2
    return np.sqrt(np.mean(squared_diffs))

# 10-point rolling RMSSD (adjust based on sampling rate)
df['RMSSD'] = df['RR'].rolling(window=10).apply(compute_rmssd, raw=True)

average_rmssd = df['RMSSD'].mean()
df['Above Avg HRV'] = df['RMSSD'] > average_rmssd


In [5]:
# Create triple y-axis layout
fig = plt.figure(figsize=(15, 5))
host = host_subplot(111, axes_class=AA.Axes)
plt.subplots_adjust(left=0.4, right=0.5)

par1 = host.twinx()      # Right axis for Heart Rate
par2 = host.twinx()      # Motion layer

# 📌 Offset heart rate axis further to the right
par1.axis["right"].toggle(all=True)
par2.axis["right"] = par2.get_grid_helper().new_fixed_axis(loc="right", axes=par2, offset=(60, 0))
par2.axis["right"].toggle(all=True)

# ⏰ Format x-axis ticks
host.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

# 🟦 Plot RMSSD on left y-axis
p1, = host.plot(df['Time'], df['RMSSD'], color='steelblue', label='HRV (RMSSD)', linewidth=1)
host.axhline(df['RMSSD'].mean(), color='gray', linestyle='--')
host.set_ylabel('HRV (ms)', color='steelblue')
host.tick_params(axis='y', labelcolor='steelblue')

# 🔴 Heart Rate on second y-axis
p2, = par1.plot(df['Time'], df['Pulse Rate'], color='crimson', linestyle='--', label='Heart Rate (BPM)', linewidth=1)
par1.set_ylabel('Heart Rate (BPM)', color='crimson')
par1.tick_params(axis='y', labelcolor='crimson')

# 🟧 Motion as third line without visible axis
p3, = par2.plot(df['Time'], df['Motion'], color='darkorange', label='Motion', alpha=0.5, linewidth=1)
par2.axis["right"].label.set_visible(False)
par2.axis["right"].major_ticklabels.set_visible(False)
par2.axis["right"].major_ticks.set_visible(False)

# 🧭 Legend
host.legend(handles=[p1, p2, p3], loc='upper left')
plt.title('HRV (RMSSD) with Heart Rate and Motion Overlay')
plt.tight_layout(pad=2.0)
#plt.show()

# Calculate static limits based on full dataset
rmssd_min, rmssd_max = df['RMSSD'].min() - 0.005, df['RMSSD'].max() + 0.005
pulse_min, pulse_max = df['Pulse Rate'].min() - 1, df['Pulse Rate'].max() + 1
motion_min, motion_max = df['Motion'].min() - 1, df['Motion'].max() + 1

# Apply fixed limits to each axis
host.set_ylim(rmssd_min, rmssd_max)
par1.set_ylim(pulse_min, pulse_max)
par2.set_ylim(motion_min, motion_max)

# Animation update function
def update(frame):
    start_time = df['Time'].iloc[0] + pd.Timedelta(seconds=HeartRateMeasuringIntervalInSeconds * frame)
    end_time = start_time + pd.Timedelta(seconds=LiveChartWindowSizeInSeconds)
    window_df = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]

    if window_df.empty:
        return p1, p2, p3

    # Update plot data
    p1.set_data(window_df['Time'], window_df['RMSSD'])
    p2.set_data(window_df['Time'], window_df['Pulse Rate'])
    p3.set_data(window_df['Time'], window_df['Motion'])

    # Adjust x-axis
    host.set_xlim(start_time, end_time)

    return p1, p2, p3

print(f'Creating Animation with Window Size: {LiveChartWindowSizeInSeconds}sec and Data Increments of {HeartRateMeasuringIntervalInSeconds}sec. Video Length: {len(df) * 1.0 / HeartRateMeasuringIntervalInSeconds / 60.0:.2f}min')

# Create animation
ani = FuncAnimation(fig, update, frames=len(df), interval=HeartRateMeasuringIntervalInSeconds * 1000)
#plt.show()

# Flag to control the loop
exporting = True

def show_progress():
    max_width = len("Generating...")  # fixed width to overwrite trailing dots
    dots = 0
    while exporting:
        message = "Generating" + "." * (dots % 4)
        padded = message.ljust(max_width)  # pad with spaces
        print(f"\r{padded}", end="", flush=True)
        dots += 1
        time.sleep(0.5)

# Start background thread
progress_thread = threading.Thread(target=show_progress)
progress_thread.start()

ani.save(f'Data/Exports/{ChartExportFileName}.mp4', writer='ffmpeg', fps=1.0 / HeartRateMeasuringIntervalInSeconds)

# Stop the progress indicator
exporting = False
progress_thread.join()

print(f'\nAnimation Exported to: {ChartExportFileName}.mp4')

plt.close(fig)

Creating Animation with Window Size: 60sec and Data Increments of 2sec. Video Length: 11.58min
Generating.  
Animation Exported to: Brett Chart.mp4


In [6]:
eeg_df = pd.read_csv(f'Data/EEG/{EEG_CSVFileName}.csv')

#O1,O2,T3,T4,O1_T3,O2_T4
eeg_df['O1'] = pd.to_numeric(eeg_df['O1'], errors='coerce')
eeg_df['O2'] = pd.to_numeric(eeg_df['O2'], errors='coerce')
eeg_df['T3'] = pd.to_numeric(eeg_df['T3'], errors='coerce')
eeg_df['T4'] = pd.to_numeric(eeg_df['T4'], errors='coerce')
eeg_df['O1_T3'] = pd.to_numeric(eeg_df['O1_T3'], errors='coerce')
eeg_df['O2_T4'] = pd.to_numeric(eeg_df['O2_T4'], errors='coerce')

#print(eeg_df)

spectrumController = SpectrumController()

def __processed_spectrum(self, spectrum, channel):
        print(spectrum)
        return
        match channel:
            case 'O1':
                self.o1Graph.update_data(spectrum)
            case 'O2':
                self.o2Graph.update_data(spectrum)
            case 'T3':
                self.t3Graph.update_data(spectrum)
            case 'T4':
                self.t4Graph.update_data(spectrum)
            case _:
                return

def __processed_waves(self, waves, channel):
     print(waves)
     return
     match channel:
        case 'O1':
            eeg_df['O1_Alpha'] = round(waves.alpha_rel * 100)
        case 'O2':
            self.o2Graph.update_data(spectrum)
        case 'T3':
            self.t3Graph.update_data(spectrum)
        case 'T4':
            self.t4Graph.update_data(spectrum)
        case _:
            return

spectrumController.processedSpectrum = __processed_spectrum
spectrumController.processedWaves = __processed_waves

#spectrumController.process_data(eeg_df)